In [16]:
# txt -> json
import re
import json

def read_text_file(file_path):
    """지정된 경로의 텍스트 파일을 읽어 내용을 반환합니다."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            # 원본 텍스트를 그대로 읽습니다. 줄바꿈 처리는 파서가 담당합니다.
            text = f.read()
        return text
    except FileNotFoundError:
        print(f"오류: 파일을 찾을 수 없습니다 - {file_path}")
        return None
    except Exception as e:
        print(f"오류: TXT 파일을 읽는 중 예외 발생 - {e}")
        return None

def parse_korean_law_to_json(text):
    """
    한국 법률 텍스트(장, 절, 조, 항, 호, 목 구조)를 JSON으로 파싱합니다.
    TXT 파일의 줄바꿈 오류와 머리글/바닥글을 처리하도록 개선되었습니다.
    """
    lines = text.strip().split('\n')

    law_data = {
        "law_title": "",
        "enforcement_date": "",
        "law_number": "",
        "revision_date": "",
        "competent_authority": "",
        "contact_info": [],
        "chapters": [],
        "addendum": []
    }

    # --- 1. Metadata Parsing ---
    metadata_lines = []
    start_line_index = 0
    # 제1장이 나올 때까지 메타데이터로 간주
    for i, line in enumerate(lines):
        # 제1장 패턴 (들여쓰기 포함)
        if re.match(r'^\s*제\s*1\s*장', line):
            start_line_index = i
            break
        metadata_lines.append(line)
    else: # 제1장을 못찾으면 제1조부터 시작
        for i, line in enumerate(lines):
             if line.strip().startswith("제1조(목적)"):
                 start_line_index = i
                 print(f"경고: '제1장'을 찾지 못함. {i+1}번째 줄 '제1조(목적)'부터 파싱을 시작합니다.")
                 break
        else:
             print("오류: 법률 내용 시작(제1조)을 찾지 못했습니다.")
             return None

    # 메타데이터 라인들을 하나로 합쳐서 파싱
    metadata_text = " ".join(metadata_lines)
    metadata_text = re.sub(r'\s+', ' ', metadata_text).strip() # 중복 공백 제거

    if "특정 금융거래정보의 보고 및 이용 등에 관한 법률" in metadata_text:
        law_data["law_title"] = "특정 금융거래정보의 보고 및 이용 등에 관한 법률"

    date_match = re.search(r'\[시행 (\d{4}\. \d{1,2}\. \d{1,2}\.)\] \[법률 (제\d+호), (\d{4}\. \d{1,2}\. \d{1,2}\.), 일부개정\]', metadata_text)
    if date_match:
        law_data["enforcement_date"] = date_match.group(1).strip()
        law_data["law_number"] = date_match.group(2).strip()
        law_data["revision_date"] = date_match.group(3).strip()

    if "개인정보보호위원회" in metadata_text:
        law_data["competent_authority"] = "개인정보보호위원회"
    
    # 연락처 정보 파싱 (개선된 정규식)
    contact_matches = re.findall(r'개인정보보호위원회\s*\((.*?)\)\s*([\d\s,-]+(?:\s*[\d,]+)*)', metadata_text)
    for match in contact_matches:
        dept = match[0].strip()
        # 전화번호 문자열에서 모든 공백(줄바꿈 포함) 제거
        phones_str = re.sub(r'\s', '', match[1])
        # 쉼표로 분리
        phones = [p.strip() for p in phones_str.split(',') if p.strip()]
        
        processed_phones = []
        base_num_prefix = ""
        for p in phones:
            if len(p) <= 4 and base_num_prefix: # 2499, 3047 등 짧은 번호
                processed_phones.append(base_num_prefix + p)
            else:
                processed_phones.append(p)
                if '-' in p: # 02-2100-2484
                    # 다음 번호를 위한 기준 번호 (예: "02-2100-")
                    base_num_prefix = p[:p.rfind('-') + 1]
                 
        law_data["contact_info"].append({"department": dept, "phone": processed_phones if len(processed_phones) > 1 else processed_phones[0]})


    # --- 2. Main Content Parsing ---
    current_chapter = None
    current_section = None
    current_article = None
    current_clause = None
    current_item = None
    current_sub_item = None

    # Regex patterns (수정됨)
    chapter_re = re.compile(r'^\s*(제\s*(\d+|[一-龥]+)\s*장)\s+(.*)')
    section_re = re.compile(r'^\s*(제\s*(\d+|[一-龥]+)\s*절)\s+(.*)')
    # non-greedy (.*?) and capture remaining text (.*)
    article_re = re.compile(r'^\s*(제\d+조(?:의\d+)?)\s*(\(.*?\))?(.*)') 
    clause_re = re.compile(r'^\s*(①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩|⑪)\s*(.*)')
    # Handle non-breaking space (\u00a0) and tabs
    item_re = re.compile(r'^[ \t\u00a0]*(\d+(?:의\d+)?)\.\s+(.*)')
    sub_item_re = re.compile(r'^[ \t\u00a0]*([가-힣])\.\s+(.*)')
    
    tag_re = re.compile(r'\s*<(개정|신설|삭제)\s+([^>]+)>') 
    trailing_info_re = re.compile(r'\s*\[(본조신설|전문개정|제목개정)\s+([^\]]+)\]')
    addendum_re = re.compile(r'^\s*부\s*칙\s*(<.*>)?')
    addendum_article_re = re.compile(r'^\s*(제\d+조)\s*(\(.*\))\s*(.*)')

    text_buffer = ""

    def clean_line(line):
        """TXT 파일 내 불필요한 머리글/바닥글 제거 (개선)"""
        line = line.strip()
        # "법제처" 또는 "국가법령정보센터"만 있는 줄
        if re.fullmatch(r'^\s*법제처\s*$', line) or re.fullmatch(r'\s*국가법령정보센터\s*$', line):
            return ""
        # "법제처 [숫자] 국가법령정보센터" 패턴
        if re.search(r'법제처\s+\d+\s+국가법령정보센터', line):
            return ""
        # "개인정보 보호법" (본문 중간에 나오는 제목)
        if re.fullmatch(r'\s*개인정보 보호법\s*', line):
             return ""
        # 페이지만 덩그러니 있는 줄
        if re.fullmatch(r'\s*\d+\s*', line):
            return ""
        return line

    def process_buffer(buffer):
        """버퍼 텍스트를 현재 구조 요소에 추가하고 버퍼 비우기 (개선)"""
        nonlocal current_sub_item, current_item, current_clause, current_article, current_addendum_article, parsing_addendum
        buffer = buffer.strip()
        if not buffer: return ""

        target = None
        if parsing_addendum:
            if current_addendum_article:
                target = current_addendum_article
            # 부칙 제목 등 (조항 시작 전) 텍스트는 일단 무시
        elif current_sub_item: target = current_sub_item
        elif current_item: target = current_item
        elif current_clause: target = current_clause
        elif current_article:
            if not current_article['clauses']:
                current_article['clauses'].append({"clause_number": None, "text": "", "items": []})
            target = current_article['clauses'][-1]
        
        if target:
            # 태그(<개정...>) 먼저 찾아서 분리
            tags_found = []
            clean_buffer = buffer
            # 태그가 줄바꿈되어도 인식하도록 re.DOTALL 사용은 위험 (여기선 한 줄 기준)
            for match in tag_re.finditer(buffer):
                tags_found.append(match.group(0).strip())
                clean_buffer = clean_buffer.replace(match.group(0), "")
            
            clean_buffer = re.sub(r'\s+', ' ', clean_buffer).strip() # 태그 제거 후 남은 텍스트 정리
            
            existing_text = target.get('text', '')
            
            if existing_text and clean_buffer:
                target['text'] = existing_text + " " + clean_buffer
            elif clean_buffer: # 기존 텍스트 없음
                target['text'] = clean_buffer
            
            # 찾은 태그들을 텍스트 뒤에 다시 붙이기
            if tags_found:
                 target['text'] = (target.get('text', '') + " " + " ".join(tags_found)).strip()

        return "" # 버퍼 비우기

    parsing_addendum = False
    current_addendum = None
    current_addendum_article = None

    # --- 실제 법 내용 파싱 (start_line_index 부터) ---
    for line in lines[start_line_index:]:
        cleaned_line = clean_line(line)
        if not cleaned_line:
            continue

        # --- 부칙 처리 ---
        addendum_match = addendum_re.match(cleaned_line)
        if addendum_match:
            text_buffer = process_buffer(text_buffer)
            parsing_addendum = True
            addendum_info = addendum_match.group(1) if addendum_match.group(1) else ""
            current_addendum = {"info": addendum_info.strip(), "articles": []}
            law_data["addendum"].append(current_addendum)
            current_chapter = current_section = current_article = current_clause = current_item = current_sub_item = None
            current_addendum_article = None
            continue

        if parsing_addendum:
            addendum_article_match = addendum_article_re.match(cleaned_line)
            if addendum_article_match:
                 text_buffer = process_buffer(text_buffer)
                 article_num = addendum_article_match.group(1)
                 article_title = addendum_article_match.group(2)
                 article_text = addendum_article_match.group(3)
                 current_addendum_article = {
                     "article_number": article_num,
                     "article_title": article_title.strip() if article_title else None,
                     "text": article_text.strip()
                 }
                 # 텍스트에 태그가 포함될 수 있으므로 process_buffer와 유사하게 처리
                 tags_found = []
                 clean_text = current_addendum_article["text"]
                 for match in tag_re.finditer(clean_text):
                      tags_found.append(match.group(0).strip())
                      clean_text = clean_text.replace(match.group(0), "")
                 clean_text = re.sub(r'\s+', ' ', clean_text).strip()
                 current_addendum_article["text"] = (clean_text + " " + " ".join(tags_found)).strip()

                 current_addendum["articles"].append(current_addendum_article)
            elif current_addendum_article:
                 text_buffer += " " + cleaned_line if text_buffer else cleaned_line
            continue

        # --- 본문 파싱 ---
        chapter_match = chapter_re.match(cleaned_line)
        section_match = section_re.match(cleaned_line)
        article_match = article_re.match(cleaned_line)
        clause_match = clause_re.match(cleaned_line)
        item_match = item_re.match(cleaned_line)
        sub_item_match = sub_item_re.match(cleaned_line)
        trailing_match = trailing_info_re.search(cleaned_line)

        # 구조 변경 시 이전 버퍼 처리
        if chapter_match or section_match or article_match or clause_match or item_match or sub_item_match:
            text_buffer = process_buffer(text_buffer)

        # 구조 처리
        if chapter_match:
            current_chapter = {"chapter_number": chapter_match.group(1).strip(), "chapter_title": chapter_match.group(3).strip(), "sections": []}
            title_tag_match = tag_re.search(current_chapter["chapter_title"])
            if title_tag_match:
                current_chapter["chapter_title"] = (current_chapter["chapter_title"].replace(title_tag_match.group(0), "").strip() + " " + title_tag_match.group(0).strip()).strip()
            law_data["chapters"].append(current_chapter)
            current_section = {"section_number": None, "section_title": None, "articles": []}
            current_chapter["sections"].append(current_section)
            current_article = current_clause = current_item = current_sub_item = None
            text_buffer = ""

        elif section_match and current_chapter:
            if current_section and current_section["section_number"] is None and not current_section["articles"]:
                current_chapter["sections"].pop()
            current_section = {"section_number": section_match.group(1).strip(), "section_title": section_match.group(3).strip(), "articles": []}
            title_tag_match = tag_re.search(current_section["section_title"])
            if title_tag_match:
                 current_section["section_title"] = (current_section["section_title"].replace(title_tag_match.group(0), "").strip() + " " + title_tag_match.group(0).strip()).strip()
            current_chapter["sections"].append(current_section)
            current_article = current_clause = current_item = current_sub_item = None
            text_buffer = ""

        elif article_match and current_section:
            article_num = article_match.group(1)
            article_title = article_match.group(2)
            remaining_line_text = article_match.group(3).strip()

            current_article = {"article_number": article_num, "article_title": article_title.strip() if article_title else None, "clauses": [], "trailing_text": ""}
            current_section["articles"].append(current_article)
            current_clause = current_item = current_sub_item = None
            
            trailing_on_article_line = trailing_info_re.search(remaining_line_text)
            if trailing_on_article_line:
                 current_article["trailing_text"] = trailing_on_article_line.group(0).strip()
                 remaining_line_text = remaining_line_text.replace(trailing_on_article_line.group(0), "").strip()

            if remaining_line_text:
                # 남은 텍스트가 항/호 인지 즉시 확인
                clause_match_remain = clause_re.match(remaining_line_text)
                item_match_remain = item_re.match(remaining_line_text)
                
                if clause_match_remain:
                    clause_num = clause_match_remain.group(1)
                    clause_text_raw = clause_match_remain.group(2).strip()
                    trailing_in_clause = trailing_info_re.search(clause_text_raw)
                    if trailing_in_clause:
                         trailing_text = trailing_in_clause.group(0).strip()
                         clause_text_clean = clause_text_raw.replace(trailing_text, "").strip()
                         current_article["trailing_text"] = (current_article.get("trailing_text","") + " " + trailing_text).strip()
                    else:
                         clause_text_clean = clause_text_raw
                    current_clause = {"clause_number": clause_num, "text": clause_text_clean, "items": []}
                    current_article["clauses"].append(current_clause)
                    text_buffer = ""
                
                elif item_match_remain:
                    if not current_clause:
                         current_clause = {"clause_number": None, "text": "", "items": []}
                         current_article["clauses"].append(current_clause)
                    item_num = item_match_remain.group(1)
                    item_text = item_match_remain.group(2).strip()
                    current_item = {"item_number": item_num, "text": item_text, "sub_items": []}
                    current_clause["items"].append(current_item)
                    text_buffer = ""

                else: # 항/호가 아닌 조항 설명 텍스트 (제1조, 제59조 등)
                    text_buffer = remaining_line_text
            else:
                text_buffer = ""

        elif clause_match and current_article:
            clause_num = clause_match.group(1)
            clause_text_raw = clause_match.group(2).strip()
            trailing_in_clause = trailing_info_re.search(clause_text_raw)
            if trailing_in_clause:
                 trailing_text = trailing_in_clause.group(0).strip()
                 clause_text_clean = clause_text_raw.replace(trailing_text, "").strip()
                 current_article["trailing_text"] = (current_article.get("trailing_text","") + " " + trailing_text).strip()
            else:
                 clause_text_clean = clause_text_raw
            current_clause = {"clause_number": clause_num, "text": clause_text_clean, "items": []}
            current_article["clauses"].append(current_clause)
            current_item = current_sub_item = None
            text_buffer = ""

        elif item_match and current_article: # 항이 있거나(current_clause) 없거나(제59조)
            if not current_clause: # 제59조 같은 케이스
                 if not current_article['clauses']: # 암묵적 항이 없으면 생성
                     current_clause = {"clause_number": None, "text": "", "items": []}
                     current_article["clauses"].append(current_clause)
                 else: # 이미 암묵적 항이 있으면(텍스트가 채워짐) 그걸 사용
                     current_clause = current_article['clauses'][-1]

            item_num = item_match.group(1)
            item_text = item_match.group(2).strip()
            current_item = {"item_number": item_num, "text": item_text, "sub_items": []}
            current_clause["items"].append(current_item)
            current_sub_item = None
            text_buffer = ""

        elif sub_item_match and current_item:
            sub_item_num = sub_item_match.group(1)
            sub_item_text = sub_item_match.group(2).strip()
            current_sub_item = {"sub_item_number": sub_item_num, "text": sub_item_text}
            current_item["sub_items"].append(current_sub_item)
            text_buffer = ""

        elif trailing_match and current_article and not (clause_match or item_match or sub_item_match):
             current_article["trailing_text"] = (current_article.get("trailing_text","") + " " + trailing_match.group(0).strip()).strip()
             cleaned_line_without_trailing = cleaned_line.replace(trailing_match.group(0), "").strip()
             if cleaned_line_without_trailing:
                 text_buffer += (" " + cleaned_line_without_trailing) if text_buffer else cleaned_line_without_trailing
             text_buffer = process_buffer(text_buffer) # 후행 정보 라인이면 즉시 처리

        # 구조 마커가 아닌 일반 텍스트 라인
        elif cleaned_line:
            text_buffer += (" " + cleaned_line) if text_buffer else cleaned_line

    # 마지막 남은 버퍼 처리
    process_buffer(text_buffer)

    # --- Cleanup ---
    for chapter in law_data["chapters"]:
        chapter["sections"] = [s for s in chapter["sections"] if s["articles"] or s["section_number"] is not None]

    return json.dumps(law_data, ensure_ascii=False, indent=2)

# --- 메인 실행 부분 ---
txt_file_path = '중대재해처벌법.txt' # <-- 여기에 실제 TXT 파일 경로를 입력하세요.

# 1. TXT 파일 읽기
raw_text = read_text_file(txt_file_path)

if raw_text:
    # 2. JSON 파싱
    parsed_json = parse_korean_law_to_json(raw_text)

    if parsed_json:
        # 3. 결과 출력
        print(parsed_json)
        
        # 파일로 저장
        try:
            with open('특정금융정보법_parsed.json', 'w', encoding='utf-8') as f:
                f.write(parsed_json)
            print("\n---")
            print("JSON 파일 저장 완료: 개인정보보호법_parsed.json")
        except Exception as e:
            print(f"\nJSON 파일 저장 실패: {e}")
    else:
        print("JSON 파싱에 실패했습니다.")
else:
    print("TXT 파일 읽기에 실패했습니다.")

{
  "law_title": "",
  "enforcement_date": "",
  "law_number": "",
  "revision_date": "",
  "competent_authority": "",
  "contact_info": [],
  "chapters": [
    {
      "chapter_number": "제1장",
      "chapter_title": "총칙",
      "sections": [
        {
          "section_number": null,
          "section_title": null,
          "articles": [
            {
              "article_number": "제1조",
              "article_title": "(목적)",
              "clauses": [
                {
                  "clause_number": null,
                  "text": "이 법은 사업 또는 사업장, 공중이용시설 및 공중교통수단을 운영하거나 인체에 해로운 원료나 제조물을 취급하면서 안전ㆍ보건 조치의무를 위반하여 인명피해를 발생하게 한 사업주, 경영책임자, 공무원 및 법인의 처벌 등을 규정함으로써 중대재해를 예방하고 시민과 종사자의 생명과 신체를 보호함을 목적으로 한다.",
                  "items": []
                }
              ],
              "trailing_text": ""
            },
            {
              "article_number": "제2조",
              "article_title": "(정의)",
              "clauses": [
                {
                  "clause_num

In [17]:
# json -> csv
import json
import csv
import re

def clean_and_join_text(clauses):
    """
    조(article)에 포함된 여러 조항(clauses)의 텍스트를 하나로 합치고,
    <개정 ...> 같은 불필요한 태그와 '삭제' 조항을 제거합니다.
    """
    full_text = []

    for clause in clauses:
        # 각 조항의 텍스트를 가져와서 태그를 제거하고 앞뒤 공백을 정리합니다.
        clause_main_text = clause.get('text', '')
        cleaned_main_text = re.sub(r'<[^>]+>', '', clause_main_text).strip()

        # 정리된 텍스트가 '삭제'이면 이 조항 전체를 건너뜁니다.
        if cleaned_main_text == '삭제':
            continue
            
        # '삭제' 조항이 아니면, 해당 조항의 모든 텍스트 부분을 리스트에 담습니다.
        text_parts = [clause.get('text', '')]
        for item in clause.get('items', []):
            text_parts.append(item.get('text', ''))
            for sub_item in item.get('sub_items', []):
                text_parts.append(sub_item.get('text', ''))
        
        # 모든 텍스트 부분을 하나의 문자열로 합칩니다.
        clause_full_text = ' '.join(filter(None, text_parts))
        
        # 합쳐진 문자열에서 태그를 최종적으로 제거합니다.
        cleaned_clause_text = re.sub(r'<[^>]+>', '', clause_full_text)
        cleaned_clause_text = re.sub(r'\[[^\]]+\]', '', cleaned_clause_text)
        
        full_text.append(cleaned_clause_text.strip())

    # 모든 유효한 조항의 내용을 하나의 문자열로 합칩니다.
    final_content = ' '.join(full_text)
    # 여러 개의 공백을 하나로 변경합니다.
    return re.sub(r'\s+', ' ', final_content).strip()

def convert_law_json_to_csv(json_file, csv_file):
    """
    법률 JSON 파일을 읽어 CSV 파일로 변환하는 메인 함수입니다.
    """
    try:
        # JSON 파일 읽기
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        law_title = data.get('law_title', 'N/A')

        # CSV 파일 쓰기
        with open(csv_file, 'w', newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f)
            
            # CSV 헤더(제목 행) 작성
            headers = ['법률명', '장 번호', '장 제목', '절 번호', '절 제목', '조 번호', '조 제목', '조 내용']
            writer.writerow(headers)

            # JSON 데이터를 순회하며 CSV 데이터 생성
            for chapter in data.get('chapters', []):
                for section in chapter.get('sections', []):
                    for article in section.get('articles', []):
                        # article_number가 없는 경우는 건너뜁니다.
                        if not article.get('article_number'):
                            continue

                        # 조(article)의 내용을 정리하여 가져옵니다.
                        content = clean_and_join_text(article.get('clauses', []))
                        
                        # 처리 후 내용이 비어있으면(예: 모든 조항이 '삭제'인 경우) CSV에 쓰지 않습니다.
                        if not content:
                            continue

                        # CSV 파일에 한 행을 작성합니다.
                        row = [
                            law_title,
                            chapter.get('chapter_number', ''),
                            chapter.get('chapter_title', ''),
                            section.get('section_number', ''),
                            section.get('section_title', ''),
                            article.get('article_number', ''),
                            article.get('article_title', ''),
                            content
                        ]
                        writer.writerow(row)

        print(f"성공: '{json_file}' 파일이 '{csv_file}' 파일로 변환되었습니다.")

    except FileNotFoundError:
        print(f"오류: '{json_file}' 파일을 찾을 수 없습니다. 파일 이름을 확인해주세요.")
    except Exception as e:
        print(f"변환 중 오류가 발생했습니다: {e}")

# --- 스크립트 실행 ---
if __name__ == "__main__":
    json_filename = '특정금융정보법_parsed.json'
    csv_filename = '특정금융정보법.csv'
    convert_law_json_to_csv(json_filename, csv_filename)

성공: '특정금융정보법_parsed.json' 파일이 '특정금융정보법.csv' 파일로 변환되었습니다.


In [18]:
# csv 합치기
import pandas as pd
import os

def merge_csv_with_pandas(file_list, output_path):
    """
    pandas 라이브러리를 사용하여 여러 개의 CSV 파일을 효율적으로 합칩니다.
    """
    # 존재하는 파일만 리스트에 담습니다.
    existing_files = [f for f in file_list if os.path.exists(f)]
    if not existing_files:
        print("오류: 합칠 수 있는 파일이 하나도 없습니다. 파일 경로를 확인해주세요.")
        return

    try:
        # 각 CSV 파일을 DataFrame으로 읽어 리스트에 담습니다.
        # (pd.read_csv(f) for f in existing_files)는 generator expression으로, 메모리를 효율적으로 사용합니다.
        df_list = (pd.read_csv(f, encoding='utf-8') for f in existing_files)
        
        # DataFrame 리스트를 위아래로 합칩니다.
        # ignore_index=True 옵션은 기존 파일들의 인덱스를 무시하고 새로 인덱스를 부여합니다.
        merged_df = pd.concat(df_list, ignore_index=True)
        
        # 합쳐진 DataFrame을 하나의 CSV 파일로 저장합니다.
        # index=False 옵션은 DataFrame의 인덱스가 파일에 열로 저장되는 것을 방지합니다.
        merged_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        
        print(f"성공: {len(existing_files)}개의 파일이 '{output_path}' 파일로 합쳐졌습니다.")

    except Exception as e:
        print(f"파일을 합치는 중 오류가 발생했습니다: {e}")


# --- 스크립트 실행 부분 ---
if __name__ == "__main__":
    # 합칠 파일들의 목록을 리스트로 만듭니다.
    files_to_merge = [
        '개인정보보호법.csv',
        '금융소비자보호법.csv',
        '아동복지법.csv',
        '자본시장법.csv',
        '전자금융거래법.csv',
        '전자증권법.csv',
        '정보통신망법.csv',
        '중대재해처벌법.csv',
        '특정금융정보법.csv'
    ]

    # 저장될 결과 파일 이름
    output_file = 'law_total.csv'

    merge_csv_with_pandas(files_to_merge, output_file)

성공: 9개의 파일이 'law_total.csv' 파일로 합쳐졌습니다.
